<a href="https://colab.research.google.com/github/AkshatHotCode/playstoreappreview_eda/blob/main/Akshat_Rajvanshi_Play_Store_App_Review_Analysis_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <b> The Play Store apps data has enormous potential to drive app-making businesses to success. Actionable insights can be drawn for developers to work on and capture the Android market. </b>

## <b> Each app (row) has values for catergory, rating, size, and more. Another dataset contains customer reviews of the android apps.</b>

## <b> Explore and analyze the data to discover key factors responsible for app engagement and success. </b>

##Introduction
In contemporary world where mobiles is one of the most necessary gadget in our life, mobiles app are everywhere as they are easy to create and can be very lucrative. Just because of thesetwo factors, the app market is developing huge day by day. Therefore in these project we will do a comprehensive exploratory data analysis of the Android App Market by comparing over 10 thousand apps in Google Play Store across various categories. We'll look for insights in the data to devise strategies to drive growth and retention.

##Data Overview
Let's breifly look at the data we have been provided, which contains of two files:


*   apps.csv : This files contains all the details of the applications on Google Play Store. There are 13 specified features available in  these dataset that describes a given app.
*   user_reviews.csv : This files have 100 most helpful reviews for each  app. Moreover,the textin each review has been preprocessed and attributed with 3 unique features: Sentiment(Positive, Neutral or Negative), Sentiment Polarity and Sentiment Subjectivity.



In [ ]:
#Importing Pandas and Numpy
import pandas as pd
import numpy as np

In [ ]:
#Importing Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#Loading csv File
file_path = "/content/drive/MyDrive/Dataset_DS/PlayStoreData.csv"
df = pd.read_csv(file_path)

In [ ]:
df.shape

(10841, 13)

In [ ]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB
